In [1]:
import pandas
import sys
import os
import json
sys.path.insert(0, os.getcwd())
from helpers import recursive_find, read_json, create_fedora_results_table

In [3]:
# Read in the experiment json files. One experiment means one package at once version
# across splices (versions of a single dependencny) and predictors
experiments = list(recursive_find("artifacts/results/extracted/fedora", "*.json"))
print('Found %s experiment library files.' % len(experiments))

Found 11310 experiment library files.


In [4]:
df = create_fedora_results_table(experiments)
display(df)

,a,b,original,changed,analysis,seconds,predictor,prediction
0,fedora-libs-35,fedora-libs-37,first/lib64/libQt5Quick.so.5.15.5,second/lib64/libQt5Quick.so.5.15.5,missing-previously-found-symbols,0.002219,symbols,True
1,fedora-libs-35,fedora-libs-37,first/lib64/libQt5Quick.so.5.15.5,second/lib64/libQt5Quick.so.5.15.5,missing-previously-found-exports,0.265112,symbols,True
2,fedora-libs-35,fedora-libs-37,first/lib64/libQt5Quick.so.5.15.5,second/lib64/libQt5Quick.so.5.15.5,abidiff,0.056751,libabigail,True
3,fedora-libs-35,fedora-libs-37,first/lib64/libQt5Quick.so.5.15.5,second/lib64/libQt5Quick.so.5.15.5,abi-compliance-tester,0.228794,abi-laboratory,False
4,fedora-libs-34,fedora-libs-37,first/lib64/libx11globalcomm.so.1.0.0,second/lib64/libx11globalcomm.so.1.0.0,missing-previously-found-symbols,0.000056,symbols,True
...,...,...,...,...,...,...,...,...
45235,fedora-libs-34,fedora-libs-36,first/lib64/kde3/kimg_dds.so,second/lib64/kde3/kimg_dds.so,abi-compliance-tester,0.08893,abi-laboratory,False
45236,fedora-libs-34,fedora-libs-35,first/lib64/libkdeinit_kcmshell.so,second/lib64/libkdeinit_kcmshell.so,missing-previously-found-symbols,0.000112,symbols,True
45237,fedora-libs-34,fedora-libs-35,first/lib64/libkdeinit_kcmshell.so,second/lib64/libkdeinit_kcmshell.so,missing-previously-found-exports,0.000023,symbols,True
45238,fedora-libs-34,fedora-libs-35,first/lib64/libkdeinit_kcmshell.so,second/lib64/libkdeinit_kcmshell.so,abidiff,0.009622,libabigail,True


In [5]:
# Symbols has two tests - we are using missing-previously-found-symbols
df = df[df["analysis"] != "missing-previously-found-exports"]

# disagree = df.groupby(["a","b", "original", "changed"])['prediction'].nunique() > 1
# disagree = pandas.DataFrame(disagree)
# Where all predictors agree
all_predictors_agree = pandas.DataFrame(df.groupby(['a', 'b', 'original', 'changed'])['prediction'].nunique() == 1).reset_index()
df_all_agree = df.merge(all_predictors_agree, on=['a', 'b', 'original', 'changed'], suffixes=["", "_all_agree"])
df_all_agree = df_all_agree[df_all_agree['prediction_all_agree']]
df_all_agree

,a,b,original,changed,analysis,seconds,predictor,prediction,prediction_all_agree
390,fedora-libs-36,fedora-libs-37,first/lib64/python3.10/site-packages/gssapi/ra...,second/lib64/python3.10/site-packages/gssapi/r...,missing-previously-found-symbols,0.000022,symbols,False,True
391,fedora-libs-36,fedora-libs-37,first/lib64/python3.10/site-packages/gssapi/ra...,second/lib64/python3.10/site-packages/gssapi/r...,abidiff,0.01013,libabigail,False,True
392,fedora-libs-36,fedora-libs-37,first/lib64/python3.10/site-packages/gssapi/ra...,second/lib64/python3.10/site-packages/gssapi/r...,abi-compliance-tester,0.088721,abi-laboratory,False,True
1224,fedora-libs-35,fedora-libs-36,first/lib64/qt5/plugins/kf5/kio/kio_help.so,second/lib64/kde3/kio_help.so,missing-previously-found-symbols,0.00006,symbols,False,True
1225,fedora-libs-35,fedora-libs-36,first/lib64/qt5/plugins/kf5/kio/kio_help.so,second/lib64/kde3/kio_help.so,abidiff,0.00932,libabigail,False,True
...,...,...,...,...,...,...,...,...,...
32956,fedora-libs-34,fedora-libs-35,first/lib64/python3.10/site-packages/gssapi/ra...,second/lib64/python3.10/site-packages/gssapi/r...,abidiff,0.009145,libabigail,False,True
32957,fedora-libs-34,fedora-libs-35,first/lib64/python3.10/site-packages/gssapi/ra...,second/lib64/python3.10/site-packages/gssapi/r...,abi-compliance-tester,0.116154,abi-laboratory,False,True
33021,fedora-libs-34,fedora-libs-35,first/lib64/qt5/plugins/kf5/kio/kio_help.so,second/lib64/kde3/kio_help.so,missing-previously-found-symbols,0.000065,symbols,False,True
33022,fedora-libs-34,fedora-libs-35,first/lib64/qt5/plugins/kf5/kio/kio_help.so,second/lib64/kde3/kio_help.so,abidiff,0.009134,libabigail,False,True


In [6]:
# Convert to wide format where analysis and predictor are columns.
df_wide = pandas.pivot(df, columns=['analysis', 'predictor'], index=['a', 'b', 'original', 'changed'], values='prediction')
df_wide

analysis                                                                                                                            missing-previously-found-symbols  \
predictor                                                                                                                                                    symbols   
a              b              original                                           changed                                                                               
fedora-libs-34 fedora-libs-35 first/lib/gcc/x86_64-redhat-linux/12/32/libitm.so  second/lib/gcc/x86_64-redhat-linux/12/32/libitm.so                          Unknown   
                              first/lib/gcc/x86_64-redhat-linux/12/32/libquad... second/lib64/libquadmath.so.0.0.0                                           Unknown   
                              first/lib/gcc/x86_64-redhat-linux/12/libasan.so    second/lib/gcc/x86_64-redhat-linux/12/32/libasa...                          Unknown   
                              first/lib/gcc/x86_64-redhat-linux/12/libatomic.so  second/lib/gcc/x86_64-redhat-linux/12/32/libato...                          Unknown   
                              first/lib/gcc/x86_64-redhat-linux/12/libgcc_s.so   second/lib/gcc/x86_64-redhat-linux/12/32/libgcc...                          Unknown   
...                                                                                                                                                              ...   
fedora-libs-36 fedora-libs-37 first/lib64/security/pam_wheel.so                  second/lib64/security/pam_wheel.so                                             True   
                              first/lib64/security/pam_xauth.so                  second/lib64/security/pam_xauth.so                                             True   
                              first/lib64/speech-dispatcher/spd_alsa.so          second/lib64/speech-dispatcher/spd_alsa.so                                     True   
                              first/lib64/speech-dispatcher/spd_libao.so         second/lib64/speech-dispatcher/spd_libao.so                                    True   
                              first/lib64/speech-dispatcher/spd_pulse.so         second/lib64/speech-dispatcher/spd_pulse.so                                    True   

analysis                                                                                                                               abidiff  \
predictor                                                                                                                           libabigail   
a              b              original                                           changed                                                         
fedora-libs-34 fedora-libs-35 first/lib/gcc/x86_64-redhat-linux/12/32/libitm.so  second/lib/gcc/x86_64-redhat-linux/12/32/libitm.so      False   
                              first/lib/gcc/x86_64-redhat-linux/12/32/libquad... second/lib64/libquadmath.so.0.0.0                       False   
                              first/lib/gcc/x86_64-redhat-linux/12/libasan.so    second/lib/gcc/x86_64-redhat-linux/12/32/libasa...      False   
                              first/lib/gcc/x86_64-redhat-linux/12/libatomic.so  second/lib/gcc/x86_64-redhat-linux/12/32/libato...      False   
                              first/lib/gcc/x86_64-redhat-linux/12/libgcc_s.so   second/lib/gcc/x86_64-redhat-linux/12/32/libgcc...      False   
...                                                                                                                                        ...   
fedora-libs-36 fedora-libs-37 first/lib64/security/pam_wheel.so                  second/lib64/security/pam_wheel.so                       True   
                              first/lib64/security/pam_xauth.so                  second/lib64/security/pam_xauth.so                       True   
                              first/lib64

In [7]:
# Join analysis/predictor columns
df_wide = df_wide.reset_index()
df_wide.columns = ["_".join(names) for names in df_wide.columns]
df_wide.columns

Index(['a_', 'b_', 'original_', 'changed_',
       'missing-previously-found-symbols_symbols', 'abidiff_libabigail',
       'abi-compliance-tester_abi-laboratory'],
      dtype='object')

In [8]:
# Get predictions only
df_predictions = df_wide[['missing-previously-found-symbols_symbols',
                    'abidiff_libabigail',
                    'abi-compliance-tester_abi-laboratory']]
df_predictions

,missing-previously-found-symbols_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
0,Unknown,False,False
1,Unknown,False,False
2,Unknown,False,False
3,Unknown,False,False
4,Unknown,False,False
...,...,...,...
11305,True,True,False
11306,True,True,False
11307,True,True,False
11308,True,True,False


In [9]:
# Filter to where predictions are either "True" or "False" only.
num_predictors = 3
df_predictions = df_predictions[(df_predictions == True).sum(axis=1) + (df_predictions == False).sum(axis=1) == num_predictors]
df_predictions

,missing-previously-found-symbols_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
5,True,True,False
9,True,True,False
10,True,True,False
11,True,True,False
12,True,True,False
...,...,...,...
11305,True,True,False
11306,True,True,False
11307,True,True,False
11308,True,True,False


In [10]:
# Correlations between predictors.
(df_predictions == True).corr()


,missing-previously-found-symbols_symbols,abidiff_libabigail,abi-compliance-tester_abi-laboratory
missing-previously-found-symbols_symbols,1.000000,0.888533,0.002383
abidiff_libabigail,0.888533,1.000000,0.002682
abi-compliance-tester_abi-laboratory,0.002383,0.002682,1.000000
